In [2]:
# import

import pandas as pd
import numpy as np
import random
import json
import glob
import pickle
from pprint import pprint

In [3]:
# test load quick draw
apple = np.load('../data/sketch_rnn/apple.npz', encoding='latin1', allow_pickle=True)
print(apple.files)
print(apple['train'][0].shape)
print(apple['train'][0])

['test', 'train', 'valid']
(43, 3)
[[-17 -18   0]
 [-19 -14   0]
 [-32  -2   0]
 [-13   5   0]
 [-17  11   0]
 [ -9  11   0]
 [ -9  48   0]
 [  3  32   0]
 [  9  15   0]
 [ 64  70   0]
 [  9  45   0]
 [ 10  10   0]
 [ 19   3   0]
 [ 12 -15   0]
 [ 22   0   0]
 [ 10   6   0]
 [  5  -2   0]
 [ 15 -34   0]
 [ 37 -37   0]
 [  5 -10   0]
 [  6 -15   0]
 [  6 -33   0]
 [  2 -37   0]
 [ -5 -13   0]
 [-12 -15   0]
 [-48 -33   0]
 [-13  -1   0]
 [-14   6   0]
 [-18  15   0]
 [ -5   8   1]
 [  3 -19   0]
 [ -1 -16   0]
 [ 27 -54   1]
 [-18  28   0]
 [  4  -5   0]
 [ 30  -9   0]
 [ 19  -1   0]
 [ 31  18   0]
 [ -8   0   0]
 [-19   7   0]
 [-25  16   0]
 [-22   1   0]
 [-18  -8   1]]


In [4]:
quickdraw_map = {}
df = pd.read_csv('../outputs/sketchyscene_quickdraw.csv')
df = df.dropna(subset=['quickdraw_label'])
for row in df.itertuples():
    quickdraw = np.load(f'../data/sketch_rnn/{row.quickdraw_label}.npz', encoding='latin1', allow_pickle=True)
    quickdraw_map[row.sketchyscene_label] = [row.quickdraw_label, quickdraw]


In [6]:
def px2int(px: str) -> int:
    return int(px.replace('px', ''))

num_draws = 16
train = []
for filename in glob.glob('../data/sketchy_scene/records/train/skenew/results1/*.json')[:num_draws]:
    with open(filename) as f:
        scene = json.load(f)
    # pprint.pprint(scene)
    draws = []
    for sketch in scene:
        if sketch['category'] in quickdraw_map:
            x = px2int(sketch['left']) + (px2int(sketch['width']) // 2)
            y = px2int(sketch['top']) + (px2int(sketch['height']) // 2)
            draw = {
                'label': quickdraw_map[sketch['category']][0],
                'sketch': random.choice(quickdraw_map[sketch['category']][1]['train']).tolist(),
                'position': [x, y],
            }
            draws.append(draw)
    train.append(draws)
with open('../outputs/sample.pkl', 'wb') as f:
    pickle.dump(train, f)

In [13]:
with open('../outputs/sample.pkl', 'rb') as f:
    sample = pickle.load(f)
print(len(sample), len(sample[0]), len(sample[0][0]['sketch']))
pprint(sample)

16 22 56
[[{'label': 'tree',
   'position': [715, 390],
   'sketch': [[-4, 52, 0],
              [-30, 88, 0],
              [-28, 54, 1],
              [143, -175, 0],
              [-3, 5, 0],
              [-7, 71, 0],
              [0, 31, 0],
              [3, 15, 0],
              [12, 29, 0],
              [42, 46, 1],
              [-123, -206, 0],
              [-32, -1, 0],
              [-27, -5, 0],
              [-10, -3, 0],
              [-16, -11, 0],
              [-11, -23, 0],
              [0, -48, 0],
              [24, -29, 0],
              [10, -7, 0],
              [17, -5, 0],
              [7, 7, 0],
              [-1, 26, 0],
              [-5, 4, 0],
              [-5, -10, 0],
              [-2, -31, 0],
              [8, -20, 0],
              [15, -24, 0],
              [10, -9, 0],
              [14, -5, 0],
              [21, 1, 0],
              [5, 3, 0],
              [8, 16, 0],
              [0, 18, 0],
              [13, -29, 0],
              [7